# <a id='toc1_'></a>[Pre Modeling](#toc0_)

"""
Created on Sun Jul  2 15:34:08 2023

@author: Nima
"""

**Table of contents**<a id='toc0_'></a>    
- [Pre Modeling](#toc1_)    
- [1. Introduction](#toc2_)    
  - [1.1. Key Questions](#toc2_1_)    
  - [1.2. Assumptions and Methods](#toc2_2_)    
  - [1.3. Setup and Data Collection](#toc2_3_)    
- [2. Transformation](#toc3_)    
  - [2.1 Dropping columns used for eda](#toc3_1_)    
  - [2.2 Transforming Date](#toc3_2_)    
- [3. Correlation](#toc4_)    
  - [3.1. Correlation Matrix](#toc4_1_)    
  - [3.2. Statistically Significance](#toc4_2_)    
  - [3.3. Plotting Correlation](#toc4_3_)    
  - [3.4 Correlation between independant variables](#toc4_4_)    
- [4. Pre-Modeling](#toc5_)    
  - [4.1. Implied Order](#toc5_1_)    
  - [4.2. One-Hot Encoding ](#toc5_2_)    
    - [4.2.1. Room Type ](#toc5_2_1_)    
    - [4.2.2. Bathroom Type ](#toc5_2_2_)    
    - [4.2.3. Neighbourhood Cleansed ](#toc5_2_3_)    
    - [4.2.4 Property Type](#toc5_2_4_)    
- [4. Multicollinearity](#toc6_)    
  - [4.1 Eigen Vector](#toc6_1_)    
  - [4.1 Variance Inflation Factor (VIF) ](#toc6_2_)    
- [5. Save Transformations](#toc7_)    
- [6. Key Findings](#toc8_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc2_'></a>[1. Introduction](#toc0_)



## <a id='toc2_1_'></a>[1.1. Key Questions](#toc0_)

This file focuses on adapting the dataset for machine learning technics. 
We will also be looking at the correlation between the variables

## <a id='toc2_2_'></a>[1.2. Assumptions and Methods](#toc0_)

- Remaining missing value or unknown were replaced with their means.

- Based on the correlation between the independant variables and between the dependant and the independent variables, the data was narrowed. 
When the correlation was higher than -0.7 or 0.7 between the independant variable, we considered it as a high correlation. We then compare the correlation between the dependant variable and the couple having multicollinearity. The member of the couple with the lowest correlation to log price was filtered from the dataset. 
Finally a multicollinearity test has been done as well.

- Categorical variables with an order were transformed within the same column. Categorical variables without an explicit order, were encoded with the one-hot methodology. The last columns of transformation was removed to avoid multicolinearity.



## <a id='toc2_3_'></a>[1.3. Setup and Data Collection](#toc0_)



In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import os
import nbformat
from nbconvert import PythonExporter
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [2]:
# Reading the configuration file
exec(open('executables/0_Config.py').read())

In [3]:
# Reading previous files
csv_path = 'outputs/2_eda_airbnb_vancouver.csv'

if os.path.isfile(csv_path):
    df_subset = pd.read_csv(csv_path)
    df_subset['first_review'] = pd.to_datetime(df_subset['first_review']) # CSV does not saved it as a date
    df_subset['last_review'] = pd.to_datetime(df_subset['last_review'])
    df_subset['host_since'] = pd.to_datetime(df_subset['host_since'])
    print('CSV file read successfully.')
else:
    exec(open('executables/2_EDA.py').read())
    print('File execution completed.')

CSV file read successfully.



# <a id='toc3_'></a>[2. Transformation](#toc0_)


## <a id='toc3_1_'></a>[2.1 Dropping columns used for eda](#toc0_)

In [4]:
# delete 'amenities' and host_verification column since we created binary variable for them in feature engineering
df_subset.drop(['amenities', 'host_verifications'], axis=1, inplace=True)
# deleting columns created for EDA. this information exists in other columns
df_subset.drop(['bedroom_category','neighborhood_mapping','availability_grp'], axis=1, inplace=True)
# deleting column replace by price
df_subset.drop(['price'], axis=1, inplace=True)


## <a id='toc3_2_'></a>[2.2 Transforming Date](#toc0_)

In [5]:
# Extract the year, month, and day into separate columns
df_subset['fr_year'] = df_subset['first_review'].dt.year
df_subset['fr_month'] = df_subset['first_review'].dt.month
df_subset['fr_day'] = df_subset['first_review'].dt.day
    
# Extract the year, month, and day into separate columns
df_subset['lr_year'] = df_subset['last_review'].dt.year
df_subset['lr_month'] = df_subset['last_review'].dt.month
df_subset['lr_day'] = df_subset['last_review'].dt.day

# Extract the year, month, and day into separate columns
df_subset['hs_year'] = df_subset['host_since'].dt.year
df_subset['hs_month'] = df_subset['host_since'].dt.month
df_subset['hs_day'] = df_subset['host_since'].dt.day

In [6]:
df_subset.drop(['host_since', 'last_review', 'first_review'], axis=1, inplace=True)


# <a id='toc4_'></a>[3. Correlation](#toc0_)


## <a id='toc4_1_'></a>[3.1. Correlation Matrix](#toc0_)

In [7]:
corr_matrix = df_subset.drop(df_subset.select_dtypes('object').columns, axis=1).corr()

# Calculate the correlation matrix
corr_matrix = df_subset.corr()

# Find the correlation values with Log Price
log_price_corr = corr_matrix.log_price

# Filter for positive and negative correlations
positive_corr = log_price_corr[log_price_corr > 0].sort_values(ascending=False).head(10)
negative_corr = log_price_corr[log_price_corr < 0].sort_values().head(3)

# Print the results
print("Columns positively correlated with 'Log Price':")
print(positive_corr)
print()
print("Columns negatively correlated with 'Log Price':")
print(negative_corr)

C:\Users\Nima\AppData\Local\Temp\ipykernel_3648\2116248742.py:4: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr_matrix = df_subset.corr()


Columns positively correlated with 'Log Price':
log_price                      1.000000
log_accommodates               0.611651
log_beds                       0.542805
log_bathrooms                  0.541409
log_bedrooms                   0.529060
amenity_dishwasher             0.270643
latitude                       0.218495
amenity_bathtub                0.190481
amenity_kitchen                0.141437
amenity_dedicated_workspace    0.137950
Name: log_price, dtype: float64

Columns negatively correlated with 'Log Price':
longitude            -0.175576
log_minimum_nights   -0.119466
amenity_heating      -0.081406
Name: log_price, dtype: float64


From the correlation matrix, the price of a property depends positvely on: accomodates, bedrooms, beds, bathrooms, type of room, but also the presence of a dishwasher among others. Longitude has also a small (negative) impact.

## <a id='toc4_2_'></a>[3.2. Statistically Significance](#toc0_)

In [8]:
# Create an empty DataFrame to store the results
df_log_price_corr = pd.DataFrame(columns=['Variable', 't-value', 'p-value'])

# Loop through the top elements with positive correlations
for element in positive_corr.index:
    tval, p = stats.pearsonr(df_subset[element], df_subset['log_price'])
    df_log_price_corr = df_log_price_corr.append({'Variable': element, 't-value': tval, 'p-value': p}, ignore_index=True)

df_log_price_corr

C:\Users\Nima\AppData\Local\Temp\ipykernel_3648\1622794926.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_log_price_corr = df_log_price_corr.append({'Variable': element, 't-value': tval, 'p-value': p}, ignore_index=True)
C:\Users\Nima\AppData\Local\Temp\ipykernel_3648\1622794926.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_log_price_corr = df_log_price_corr.append({'Variable': element, 't-value': tval, 'p-value': p}, ignore_index=True)
C:\Users\Nima\AppData\Local\Temp\ipykernel_3648\1622794926.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_log_price_corr = df_log_price_corr.append({'Variable': element, 't-value': tval, 'p-value': p}, ignore_index=True)
C:\Users\Nima\AppData\Local\Temp\ipykernel_364

,Variable,t-value,p-value
0,log_price,1.000000,0.000000e+00
1,log_accommodates,0.611651,0.000000e+00
2,log_beds,0.542805,0.000000e+00
3,log_bathrooms,0.541409,0.000000e+00
4,log_bedrooms,0.529060,0.000000e+00
5,amenity_dishwasher,0.270643,3.521522e-100
6,latitude,0.218495,4.279325e-65
7,amenity_bathtub,0.190481,1.268880e-49
8,amenity_kitchen,0.141437,6.588323e-28
9,amenity_dedicated_workspace,0.137950,1.287717e-26


p values are smaller than 0.05, so all the covariance with log price and the variables are significative



## <a id='toc4_3_'></a>[3.3. Plotting Correlation](#toc0_)

In [9]:
# Mask for upper-triangular
mask = np.triu(corr_matrix)

plt.figure(figsize=(100,150))
sns.heatmap(corr_matrix.round(1), annot=True, vmax=1, vmin=-1, cmap='coolwarm', mask=mask, center=0)
plt.title("Correlation Matrix",size=15, weight='bold')

# Save the plot as a JPEG file
plt.savefig('images/correlation_matrix.jpeg', format='jpeg')

# Close the figure to prevent it from being displayed in the notebook
plt.close()


## <a id='toc4_4_'></a>[3.4 Correlation between independant variables](#toc0_)

In [10]:
# Calculate the correlation matrix
corr_matrix = df_subset.corr()

# Keep variable with corr greater than 0.7 or less than -0.7
multi_col = (corr_matrix.abs() > 0.7) & (corr_matrix != 1)

# Use the multi_col to filter the correlation matrix
high_corr_matrix = corr_matrix[multi_col]

# Create a dataframe from the filtered correlation matrix
high_corr_df = pd.DataFrame(high_corr_matrix.stack()).reset_index()
high_corr_df.columns = ['Variable 1', 'Variable 2', 'Correlation']

# Sort the values in descending order based on the 'Correlation' column
high_corr_df = high_corr_df.sort_values(by='Correlation', ascending=False)

# Remove every second line
high_corr_df = high_corr_df.iloc[::2]

# Add new columns for log_price and Variable 1 correlation
high_corr_df['log_price_and_variable1'] = high_corr_df.apply(lambda row: corr_matrix.loc['log_price', row['Variable 1']], axis=1)

# Add new columns for log_price and Variable 2 correlation
high_corr_df['log_price_and_variable2'] = high_corr_df.apply(lambda row: corr_matrix.loc['log_price', row['Variable 2']], axis=1)

# Compare absolute values and display the variable with the lowest absolute correlation
high_corr_df['var_with_lowest_logprice_corr'] = np.where(
    np.abs(high_corr_df['log_price_and_variable1']) > np.abs(high_corr_df['log_price_and_variable2']),
    high_corr_df['Variable 2'],
    high_corr_df['Variable 1']
)

C:\Users\Nima\AppData\Local\Temp\ipykernel_3648\91228853.py:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr_matrix = df_subset.corr()


In [11]:
# Display the dataframe
high_corr_df

,Variable 1,Variable 2,Correlation,log_price_and_variable1,log_price_and_variable2,var_with_lowest_logprice_corr
25,lr_year,fr_year,0.998788,-0.071905,-0.069214,fr_year
1,log_host_total_listings_count,log_host_listings_count,0.954959,-0.030559,-0.024788,log_host_listings_count
11,log_availability_60,log_availability_90,0.952039,0.114963,0.106455,log_availability_90
8,log_availability_30,log_availability_60,0.885143,0.130043,0.114963,log_availability_60
23,log_reviews_per_month,log_number_of_reviews_ltm,0.863279,0.011920,0.032028,log_reviews_per_month
6,log_beds,log_accommodates,0.834067,0.542805,0.611651,log_beds
13,log_availability_90,log_availability_30,0.804621,0.106455,0.130043,log_availability_90
17,log_availability_365,log_availability_90,0.772886,0.115429,0.106455,log_availability_90
18,log_number_of_reviews,log_number_of_reviews_ltm,0.762178,-0.052225,0.032028,log_number_of_reviews_ltm
5,log_bedrooms,log_beds,0.744135,0.529060,0.542805,log_bedrooms


In [12]:
# Get the distinct values from the columns_to_drop list
columns_to_drop = list(set(high_corr_df['var_with_lowest_logprice_corr'].tolist()))

# Drop the columns from df_subset
df_subset = df_subset.drop(columns=columns_to_drop)

One of the hypothesis of a linear regression is to avoid collinearity.
When two variables have a collinearity, we removed one of them based on the one having the lower correlation with the dependant variable.


# <a id='toc5_'></a>[4. Pre-Modeling](#toc0_)

In this section, we will perform some transformation to adapt to machine learning's requirement for categorical variables


## <a id='toc5_1_'></a>[4.1. Implied Order](#toc0_)

In [13]:
# Label categorical variables with an implied order:
df_subset.host_response_time.unique()    
category_order = ['unknown', 'within an hour', 'within a few hours', 'within a day', 'a few days or more']

# Create a dictionary with category_order as keys and their corresponding indices as values
category_order = {category_order[i]: i for i in range(len(category_order))}

# Replace the keys in df_subset['host_response_time'] column with the corresponding values
df_subset['host_response_time_order'] = df_subset['host_response_time'].map(category_order)

# Reference table for meaning of host_response_time_order    
df_host_response=df_subset[['host_response_time_order','host_response_time']].drop_duplicates()

#Drop original column column
df_subset = df_subset.drop('host_response_time', axis=1)




## <a id='toc5_2_'></a>[4.2. One-Hot Encoding ](#toc0_)

Label Encoding collapses the categorical variable into a single numerical column, potentially leading to information loss.

One-Hot Encoding preserves all the information by creating separate binary columns,  but it can increase the dimensionality of the dataset. 

The trade-off between information loss > dimensionality.

Therefore I did a One-Hot for encoding the columns when there was no order in the catgorical column


### <a id='toc5_2_1_'></a>[4.2.1. Room Type ](#toc0_)

In [14]:
# Create dummy variables from room_type
dummy_df = pd.get_dummies(df_subset.room_type, prefix='room_type')
# Lowercase the values, Replace '/' with '_or_', Replace spaces with '_'
dummy_df.columns = dummy_df.columns.str.lower().str.replace('/', '_or_').str.replace(' ', '_')
# Convert True/False values to 1/0
dummy_df = dummy_df.astype(int)
# Concatenate the dummy variables with the original DataFrame
df_subset = pd.concat([df_subset, dummy_df], axis=1)
#Drop original column
df_subset = df_subset.drop('room_type', axis=1)
# Drop one of the encoding column
df_subset = df_subset.iloc[:, :-1]


### <a id='toc5_2_2_'></a>[4.2.2. Bathroom Type ](#toc0_)

In [15]:
# Create dummy variables from bathroom_type
dummy_df = pd.get_dummies(df_subset.bathroom_type, prefix='bathroom_is')
# Convert True/False values to 1/0
dummy_df = dummy_df.astype(int)
# Concatenate the dummy variables with the original DataFrame
df_subset = pd.concat([df_subset, dummy_df], axis=1)
#Drop original column
df_subset = df_subset.drop('bathroom_type', axis=1)
# Drop one of the encoding column
df_subset = df_subset.iloc[:, :-1]


### <a id='toc5_2_3_'></a>[4.2.3. Neighbourhood Cleansed ](#toc0_)

In [16]:
#Creating dummies by neighbourdhood_cleansed
dummy_df = pd.get_dummies(df_subset.neighbourhood_cleansed, prefix='ngb')
# Lowercase the values, Replace '-' with '_', Replace spaces with '_'
dummy_df.columns = dummy_df.columns.str.lower().str.replace('-', '_').str.replace(' ', '_')
# Convert True/False values to 1/0
dummy_df = dummy_df.astype(int)
# Concatenate the dummy variables with the original DataFrame
df_subset = pd.concat([df_subset, dummy_df], axis=1)
#Drop column
df_subset = df_subset.drop('neighbourhood_cleansed', axis=1)
# Drop one of the encoding column
df_subset = df_subset.iloc[:, :-1]


### <a id='toc5_2_4_'></a>[4.2.4 Property Type](#toc0_)

In [17]:
#Creating dummies by neighbourdhood_cleansed
dummy_df = pd.get_dummies(df_subset.property_type)
# Lowercase the values, Replace '-' with '_', Replace spaces with '_'
dummy_df.columns = dummy_df.columns.str.lower().str.replace('-', '_').str.replace(' ', '_')
# Convert True/False values to 1/0
dummy_df = dummy_df.astype(int)
# Concatenate the dummy variables with the original DataFrame
df_subset = pd.concat([df_subset, dummy_df], axis=1)
#Drop column
df_subset = df_subset.drop('property_type', axis=1)
# Drop one of the encoding column
df_subset = df_subset.iloc[:, :-1]


# <a id='toc6_'></a>[4. Multicollinearity](#toc0_)


## <a id='toc6_1_'></a>[4.1 Eigen Vector](#toc0_)

In [18]:
# Calculate the eigenvalues and eigenvectors of the correlation matrix
multicollinearity, V = np.linalg.eig(corr_matrix)

# Set the threshold for multicollinearity (e.g., 0.1)
multicollinearity_threshold = 0.1

# Filter eigenvalues based on the threshold
multicollinear_eigenvalues = [eig for eig in multicollinearity if eig > multicollinearity_threshold]

# Round eigenvalues to 4 decimal places
multicollinear_eigenvalues = np.round(multicollinear_eigenvalues, 4)

# Convert eigenvalues to regular decimal format (no exponential notation)
multicollinear_eigenvalues = ["{:.4f}".format(eig) for eig in multicollinear_eigenvalues]

# Print the updated eigenvalues
print("Eigenvalues with multicollinearity:")
print(multicollinear_eigenvalues)

Eigenvalues with multicollinearity:
['10.6779', '4.8003', '3.8406', '3.4818', '2.7168', '2.0412', '1.8513', '1.7468', '1.5721', '1.4196', '1.3385', '1.2575', '1.1388', '1.0601', '1.0542', '0.1112', '1.0284', '1.0109', '1.0049', '0.9953', '0.9769', '0.9666', '0.1279', '0.1472', '0.1539', '0.9343', '0.9211', '0.8917', '0.8550', '0.8585', '0.8152', '0.7967', '0.7900', '0.7662', '0.7549', '0.7486', '0.7396', '0.2281', '0.2418', '0.2702', '0.2673', '0.2898', '0.7077', '0.3104', '0.6885', '0.6793', '0.6549', '0.6514', '0.6353', '0.3333', '0.3511', '0.3745', '0.3766', '0.3968', '0.4094', '0.4104', '0.4310', '0.6056', '0.4523', '0.5862', '0.5694', '0.5592', '0.5468', '0.5227', '0.5156', '0.4966', '0.4888', '0.4834']


From the Eigenvalues, we have multiple cases of multicollinearity (when values are bigger than 0.01). We will investigate the multicollinearity with a different method to gather more insights


## <a id='toc6_2_'></a>[4.2 Variance Inflation Factor (VIF) ](#toc0_)

In [19]:
# Create an empty DataFrame to store the VIF results
vif_df = pd.DataFrame()

# Iterate over each column in the dataset
for idx, column in enumerate(df_subset.columns):
    try:
        # Calculate the VIF for the current variable
        vif = variance_inflation_factor(df_subset.values, idx)
        # Append the results to the DataFrame
        vif_df = vif_df.append({'Variable': column, 'VIF': vif}, ignore_index=True)
    except TypeError:
        print(f"Error calculating VIF for column: {column}")


C:\Users\Nima\AppData\Local\Temp\ipykernel_3648\1397070018.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  vif_df = vif_df.append({'Variable': column, 'VIF': vif}, ignore_index=True)
C:\Users\Nima\AppData\Local\Temp\ipykernel_3648\1397070018.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  vif_df = vif_df.append({'Variable': column, 'VIF': vif}, ignore_index=True)


C:\Users\Nima\AppData\Local\Temp\ipykernel_3648\1397070018.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  vif_df = vif_df.append({'Variable': column, 'VIF': vif}, ignore_index=True)
C:\Users\Nima\AppData\Local\Temp\ipykernel_3648\1397070018.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  vif_df = vif_df.append({'Variable': column, 'VIF': vif}, ignore_index=True)
C:\Users\Nima\AppData\Local\Temp\ipykernel_3648\1397070018.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  vif_df = vif_df.append({'Variable': column, 'VIF': vif}, ignore_index=True)
C:\Users\Nima\AppData\Local\Temp\ipykernel_3648\1397070018.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future versio

In [20]:
# Set the float format to avoid exponential notation
pd.options.display.float_format = '{:.2f}'.format

vif_df[vif_df['VIF']>5]

,Variable,VIF
6,latitude,14.79
7,longitude,19.91
64,room_type_entire_home_or_apt,256608337.72
67,ngb_downtown,36.65
68,ngb_downtown_eastside,12.87
71,ngb_grandview_woodland,14.28
72,ngb_hastings_sunrise,17.58
73,ngb_kensington_cedar_cottage,18.35
75,ngb_killarney,8.53
76,ngb_kitsilano,7.60


As we can see, there are some multicollinearity issues. We can expect our linear modeling prediction to be weak because of it. However multicollinearity is tolerated - to a certain extend - for the more advances modeling we will be using. Therefore, we will not narrowing more the data. Instead we will regularization methods like Ridge Regression for the linear regression


# <a id='toc7_'></a>[5. Save Transformations](#toc0_)

Save CSV

In [21]:
df_subset.to_csv('outputs/3_pm_airbnb_vancouver.csv', index=False)

Save Python Script

In [22]:
if __name__ == '__main__':
    notebook_file_path = '3_PreModeling.ipynb'
    python_file_path = 'executables/3_PreModeling.py'

    convert_notebook_to_python(notebook_file_path, python_file_path)


# <a id='toc8_'></a>[6. Key Findings](#toc0_)

There is some multicollinearity between our dependent variables, which will affect the linear modeling prediction. It's therefore required to use more advances modeling, which cope better with this problem.